<a href="https://colab.research.google.com/github/kamrulhasan04947/mlprac/blob/main/classworkdeephybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import numpy as np
import seaborn as sns
from keras.applications.vgg16 import VGG16
from google.colab.patches import cv2_imshow

In [ ]:
size = 244
path = '/content/drive/MyDrive/dataset/datas/train/*'

In [ ]:
class ImageData:
  def __init__(self, size, path):
    self.size = size
    self.path = path

  def segmentation(self , img):
        img = cv2.GaussianBlur(img, (5,5),0)
        kernel = np.ones((3, 3), np.uint8)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, threshold = cv2.threshold(gray , 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        opening = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel, iterations=2)
        sure_bg = cv2.dilate(opening, kernel, iterations=3)
        dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
        ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg, sure_fg)
        ret, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0
        cv2.watershed(img, markers)
        colored_markers = np.zeros_like(img)
        colored_markers[markers == -1] = [255, 0, 0]  # Boundaries in blue color
        # Display the segmented image
        segmented_image = cv2.addWeighted(img, 0.7, colored_markers, 0.3, 0)
        return  segmented_image

  def Imageprocessor(self):
    images = []
    labels = []
    for directory_path in glob.glob(self.path):
      for img_path in glob.glob(os.path.join(directory_path + "/*")):
          imgClsass = img_path.split('/')[-2]
          img = self.segmentation(cv2.imread(img_path, cv2.IMREAD_COLOR))
          img = cv2.resize(img, (self.size, self.size))
          images.append(img)
          labels.append(imgClsass)
    return images, labels

In [ ]:
data =  ImageData(size, path).Imageprocessor()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
#conberting to arra to data
image = np.array(data[0])
label = le.fit_transform(np.array(data[1]))

In [ ]:
x_train , x_test, y_train, y_test = train_test_split(image , label, test_size=.20, random_state = 42)

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
print(f'x_train={len(x_train)} , x_test ={len(x_test)}, y_train = {len(y_train)}, y_test= {len(y_test)}')

x_train=64 , x_test =16, y_train = 64, y_test= 16


In [ ]:
from keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
print(f'x_train={len(x_train)} , x_test ={len(x_test)}, y_train = {len(y_train_one_hot)}, y_test= {len(y_test_one_hot )}')

x_train=64 , x_test =16, y_train = 64, y_test= 16


In [ ]:
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(244, 244, 3))

In [ ]:
x = Flatten()(VGG_model.output)
prediction = Dense(4, activation='softmax')(x)
model = Model(inputs=VGG_model.input, outputs=prediction)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0   

In [ ]:
feature_extractor=VGG_model.predict(x_train)

2/2 [==============================] - 31s 14s/step


In [ ]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)
features

array([[0.19871095, 0.        , 0.        , ..., 0.        , 0.52602243,
        0.        ],
       [0.3381247 , 0.        , 0.        , ..., 0.        , 0.5165814 ,
        0.        ],
       [0.11107308, 0.        , 0.        , ..., 0.        , 0.56529784,
        0.        ],
       ...,
       [0.18067533, 0.        , 0.        , ..., 0.        , 0.5386395 ,
        0.        ],
       [0.16044277, 0.        , 0.        , ..., 0.        , 0.5552373 ,
        0.        ],
       [0.18836248, 0.        , 0.        , ..., 0.        , 0.553109  ,
        0.        ]], dtype=float32)

In [ ]:
x_test = x_test.reshape(feature_extractor.shape[0], -1)

In [ ]:
features.shape

(64, 25088)

In [ ]:
x_test.shape

(64, 44652)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
dt = DecisionTreeClassifier(criterion='entropy', random_state=10)
modelfit = dt.fit(features, y_train_one_hot)
prediction = modelfit.predict(x_test)
report = classification_report(y_test_one_hot , prediction)

ValueError: ignored

In [ ]:
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20

   micro avg       1.00      1.00      1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80
 samples avg       1.00      1.00      1.00        80



In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
r = model.fit(
  train_generator,
  validation_data= test_data_dir,
  epochs=5,
  steps_per_epoch=len( train_generator),
  validation_steps=len(test_data_dir)
)

Epoch 1/5
2/2 [==============================] - 194s 99s/step - loss: 2.8821 - accuracy: 0.3750 - val_loss: 1.0596 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 199s 107s/step - loss: 1.4580 - accuracy: 0.2500 - val_loss: 4.0656 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 183s 95s/step - loss: 1.8716 - accuracy: 0.3594 - val_loss: 0.9281 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 184s 100s/step - loss: 1.3438 - accuracy: 0.3125 - val_loss: 1.5891 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 189s 105s/step - loss: 2.2587 - accuracy: 0.3906 - val_loss: 1.3550 - val_accuracy: 0.0000e+00
